# Penjelasan Dataset & Tugas Klasifikasi
Dataset ini terdiri dari data performa ETF dan reksa dana yang beroperasi di Eropa. Saya akan melakukan tugas klasifikasi untuk memodelkan klasifikasi pada fitur 'rating'

# Import Dataset & Libraries

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from collections import Counter
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import PowerTransformer
from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE, SVMSMOTE, ADASYN
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression, f_classif
from sklearn.preprocessing import RobustScaler
from numpy import array
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import classification_report

In [ ]:
url = 'https://raw.githubusercontent.com/fauzanrrizky/datsci-datasets/main/EIM_Train.csv'
df = pd.read_csv(url)

# Functions

Fungsi-fungsi penting yang nantinya akan digunakan

In [ ]:
def evaluate_classifier_performance(prediction, y_test):
    # Informasi evaluasi secara compact
    print("Hasil Evaluasi berdasarkan classification report \n\n%s\n" % (classification_report(y_test, prediction,zero_division=0)))
    print()
    print("Confusion Matrix")
    print()
    y_actual = pd.Series(np.array(y_test), name = "actual")
    y_pred = pd.Series(np.array(prediction), name = "prediction")
    df_confusion = pd.crosstab(y_actual, y_pred)
    display(df_confusion)
    print()
    print()

    print("Butuh informasi lebih lengkap? silakan simak di bawah ini : ")
    print('Accuracy Average:', accuracy_score(y_test, prediction))
    print('F1 Macro Average:', f1_score(y_test, prediction, average='macro'))
    print('F1 Micro Average:', f1_score(y_test, prediction, average='micro'))
    print('Precision Macro Average:', precision_score(y_test, prediction, average='macro',zero_division=0))
    print('Precision Micro Average:', precision_score(y_test, prediction, average='micro',zero_division=0))
    print('Recall Macro Average:', recall_score(y_test, prediction, average='macro',zero_division=0))
    print('Recall Micro Average:', recall_score(y_test, prediction, average='micro',zero_division=0))
    print()

In [ ]:
# Missing Values
def get_missing_info(df):

    missing_values = df.isnull().sum()
    missing_percentage = (df.isnull().sum() / len(df)) * 100

    missing_info = pd.DataFrame({
        'Column': missing_values.index,
        'Missing Values': missing_values.values,
        'Missing Values Percentage': missing_percentage.values
    })

    missing_info = missing_info.sort_values(by='Missing Values Percentage', ascending=False)
    return missing_info

# Preprocessing
Tahapan preprocessing mulai dari penanganan data duplikat, data hilang, outliers, encoding & splitting data, normalisasi, feature selection, dan imbalance handling

## Data Duplikat

In [ ]:
# Duplicated Data Train
duplikat_train = df[df.duplicated()]
print(duplikat_train)

Empty DataFrame
Columns: [ticker, category, dividend_frequency, equity_style, equity_size, equity_size_score, price_prospective_earnings, price_book_ratio, price_sales_ratio, price_cash_flow_ratio, dividend_yield_factor, long_term_projected_earnings_growth, historical_earnings_growth, sales_growth, cash_flow_growth, book_value_growth, roa, roe, roic, bond_interest_rate_sensitivity, bond_credit_quality, average_coupon_rate, average_credit_quality, modified_duration, effective_maturity, asset_stock, asset_bond, asset_cash, asset_other, sector_basic_materials, sector_consumer_cyclical, sector_financial_services, sector_real_estate, sector_consumer_defensive, sector_healthcare, sector_utilities, sector_communication_services, sector_energy, sector_industrials, sector_technology, market_cap_giant, market_cap_large, market_cap_medium, market_cap_small, market_cap_micro, credit_aaa, credit_aa, credit_a, credit_bbb, credit_bb, credit_b, credit_below_b, credit_not_rated, holdings_n_stock, holdi

Tidak ada data duplikat

## Missing Values

In [ ]:
result_missing_info = get_missing_info(df)
print(result_missing_info)

                             Column  Missing Values  Missing Values Percentage
23                modified_duration           20692                  92.292596
24               effective_maturity           20529                  91.565566
20              bond_credit_quality           20133                  89.799286
19   bond_interest_rate_sensitivity           20133                  89.799286
52                 credit_not_rated           19897                  88.746655
..                              ...             ...                        ...
10            dividend_yield_factor               0                   0.000000
5                 equity_size_score               0                   0.000000
4                       equity_size               0                   0.000000
3                      equity_style               0                   0.000000
116                          rating               0                   0.000000

[117 rows x 3 columns]


Buat treshold missing values sebanyak 50%
Sayamm Menetapkan missing values treshold sebesar 50% karensayami meyakini data yang hilang di atas 50% sudah kehilangan sebagian besar informasinya. Menyimpan fitur tersebut mungkin hanya akan menambah noise dan tidak memberikan insight untuk membuat model prediktif

In [ ]:
# Tentukan batas persentase missing values
threshold_percentage = 50

# Pilih kolom-kolom yang memiliki missing values di atas threshold
columns_to_drop = result_missing_info[result_missing_info['Missing Values Percentage'] > threshold_percentage]['Column']

# Drop kolom-kolom tersebut dari DataFrame
df = df.drop(columns=columns_to_drop)

Selanjutnya, Saya mencari tahu dan membagi tipe data (numerical & categorical) untuk melakukan penanganan yang berbeda pada bagian imputasi data

> Indented block



In [ ]:
# numeric_cols = df.select_dtypes(include=['number'])
# categorical_cols = df.select_dtypes(exclude=['number'])

numerical_cols = list(df.select_dtypes(include=['number']).columns)
categorical_cols = list(df.select_dtypes(exclude=['number']).columns)

# Remove fitur ticker(identifier) dan rating(label feature)
categorical_cols.remove('ticker')
numerical_cols.remove('rating')

Selanjutnya, dilakukan imputasi Missing Values

Disini, Saya melakukan imputasi data dengan cara mengimputasi dengan nilai mean untuk kolom numerikal, dan nilai modus untuk kolom kategorikal. Namun, ada pengecualian untuk kolom sectors = ['sector_basic_materials','sector_consumer_cyclical', 'sector_financial_services',
             'sector_real_estate', 'sector_consumer_defensive', 'sector_healthcare', 'sector_utilities',
             'sector_communication_services', 'sector_energy', 'sector_industrials', 'sector_technology'].

Saya mengisi data null untuk kolom 'sector_*' dengan 0, Alasan: Null disini artinya investment management tidak menginvestasikan uangnya di sektor tersebut.

In [ ]:
sectors = ['sector_basic_materials','sector_consumer_cyclical', 'sector_financial_services',
             'sector_real_estate', 'sector_consumer_defensive', 'sector_healthcare', 'sector_utilities',
             'sector_communication_services', 'sector_energy', 'sector_industrials', 'sector_technology']

# Mengimputasi nilai 0 untuk missing value pada kolom-kolom sektor
for i in sectors:
    df[i].fillna(0, inplace=True)

# Mengimputasi nilai mean untuk missing value pada kolom numerik
for i in numerical_cols:
    mean_value = df[i].mean()
    df[i].fillna(mean_value, inplace=True)

# Mengimputasi nilai mean untuk missing value pada kolom kategorikal
for i in categorical_cols:
    mode_value = df[i].mode()[0]
    df[i].fillna(mode_value, inplace=True)

# Tampilkan DataFrame yang telah dibersihkan
result_missing_info = get_missing_info(df)
print(result_missing_info.to_string(index=False))

                             Column  Missing Values  Missing Values Percentage
                             ticker               0                        0.0
                    shareclass_size               0                        0.0
                   fund_return_2015               0                        0.0
                   fund_return_2016               0                        0.0
                   fund_return_2017               0                        0.0
                   fund_return_2018               0                        0.0
                   fund_return_2019               0                        0.0
        fund_trailing_return_5years               0                        0.0
        fund_trailing_return_3years               0                        0.0
           fund_trailing_return_ytd               0                        0.0
                          fund_size               0                        0.0
                 fund_size_currency               0 

Sudah tidak ada missing values

## Outliers

In [ ]:
# Check Outliers
# Variabel untuk outliers
Q3 = df.select_dtypes(include=['number']).quantile(0.75)
Q1 = df.select_dtypes(include=['number']).quantile(0.25)
IQR = Q3.subtract(Q1)

Jumlah_Outlier0 = []
pd.set_option('display.max_columns', None)
for col in df.select_dtypes(np.number).columns:
    outliers = ((df[col] < (Q1[col] - 1.5 * IQR[col])) | (df[col] > (Q3[col] + 1.5 * IQR[col]))).sum()
    Jumlah_Outlier0.append([col, outliers, outliers/len(df)])

indexOutput0 = list(range(0, len(Jumlah_Outlier0)))

pd.DataFrame(Jumlah_Outlier0, columns=['Column', 'Outlier','persentase'],index=indexOutput0).sort_values(by=['persentase'], ascending = False,ignore_index=True)

,Column,Outlier,persentase
0,asset_bond,5384,0.240143
1,asset_stock,4672,0.208385
2,involvement_palm_oil,4446,0.198305
3,fund_return_2015,4118,0.183675
4,asset_other,3848,0.171632
...,...,...,...
88,quarters_down,119,0.005308
89,ongoing_cost,65,0.002899
90,management_fees,61,0.002721
91,market_cap_giant,43,0.001918


Terlihat banyak kolom yang mengandung outliers, dengan 'asset_bond' dengan outliers tertinggi sebesar 24% dari keseluruhan data.

## Encoding & Splitting Data

In [ ]:
# Encoding
le = LabelEncoder()

df_encoded = df.copy()
for col in categorical_cols:
    df_encoded[col] = le.fit_transform(df[col])

# Lakukan encoding dengan manual (mapping)
# Saya melakukan mapping kolom rating untuk mulai dari 0 karena, beberapa model klasifikasi membutuhkan class yang dimulai dari 0 (bukan 1)
rating_map = {
    1:0,
    2:1,
    3:2,
    4:3,
    5:4
}
df_encoded['rating'] = df_encoded['rating'].map(rating_map)

df_encoded.head()

,ticker,category,equity_style,equity_size,equity_size_score,price_prospective_earnings,price_book_ratio,price_sales_ratio,price_cash_flow_ratio,dividend_yield_factor,long_term_projected_earnings_growth,historical_earnings_growth,sales_growth,cash_flow_growth,book_value_growth,roa,roe,roic,asset_stock,asset_bond,asset_cash,asset_other,sector_basic_materials,sector_consumer_cyclical,sector_financial_services,sector_real_estate,sector_consumer_defensive,sector_healthcare,sector_utilities,sector_communication_services,sector_energy,sector_industrials,sector_technology,market_cap_giant,market_cap_large,market_cap_medium,market_cap_small,market_cap_micro,holdings_n_stock,ongoing_cost,management_fees,environmental_score,social_score,governance_score,sustainability_score,sustainability_rank,sustainability_percentage_rank,involvement_abortive_contraceptive,involvement_alcohol,involvement_animal_testing,involvement_controversial_weapons,involvement_gambling,involvement_gmo,involvement_military_contracting,involvement_nuclear,involvement_palm_oil,involvement_pesticides,involvement_small_arms,involvement_thermal_coal,involvement_tobacco,nav_per_share_currency,nav_per_share,shareclass_size_currency,shareclass_size,fund_size_currency,fund_size,fund_trailing_return_ytd,fund_trailing_return_3years,fund_trailing_return_5years,fund_return_2019,fund_return_2018,fund_return_2017,fund_return_2016,fund_return_2015,fund_return_2020_q3,fund_return_2020_q2,fund_return_2020_q1,fund_return_2019_q4,fund_return_2019_q3,fund_return_2019_q2,fund_return_2019_q1,fund_return_2018_q4,fund_return_2018_q3,fund_return_2018_q2,fund_return_2018_q1,fund_return_2017_q4,fund_return_2017_q3,fund_return_2017_q2,fund_return_2017_q1,fund_return_2016_q4,fund_return_2016_q3,fund_return_2016_q2,fund_return_2016_q1,fund_return_2015_q4,fund_return_2015_q3,fund_return_2015_q2,fund_return_2015_q1,quarters_up,quarters_down,rating
0,0P000153IG,92,2,0,248.25,10.66,1.00,0.92,6.19,4.13,8.66,3.060000,4.72,0.16,2.75,6.38,14.04,11.80,98.55,0.00,1.45,0.00,14.17,2.35,18.69,0.00,4.31,9.30,0.00,9.28,7.74,23.95,10.21,30.75,39.04,16.80,10.69,2.72,45.0,1.136168,0.852358,6.16,8.72,8.73,24.38,1.0,89.0,9.67,2.86,11.77,0.00,0.00,0.00,0.00,1.37,0.0,0.00,0.00,0.00,0.00,7,241.84,10,261930000.0,9,6.566210e+09,1.40,2.08,10.280000,18.51,-11.31,15.240000,28.740000,2.064887,5.66,18.05,-24.50,2.42,-0.41,8.18,7.40,-14.03,5.37,5.56,-7.25,3.53,1.790000,1.950000,7.250000,6.060000,14.750000,1.85000,3.860000,8.010000,-8.180000,-4.630000,7.171196,16,6,2
1,F00000OUR6,186,0,1,124.69,23.06,1.97,1.70,12.07,1.30,9.34,14.300000,7.55,8.56,9.59,7.36,14.61,11.33,96.81,0.57,2.63,-0.01,8.17,14.42,18.82,0.00,2.96,15.34,1.59,0.00,0.00,17.32,21.39,1.14,0.00,66.33,32.53,0.00,57.0,0.840000,0.750000,5.31,8.85,7.16,26.08,2.0,78.0,3.93,0.00,14.93,0.00,0.00,0.00,4.91,0.00,0.0,2.38,0.00,1.59,0.00,16,3.96,14,17010000.0,12,7.717900e+08,1.65,6.66,12.090000,25.39,-5.72,1.740000,41.980000,7.120000,0.89,17.82,-24.09,0.12,4.68,8.60,10.16,-16.02,6.56,11.32,-5.35,5.29,-1.810000,-1.710000,0.120000,11.940000,8.640000,9.40000,6.720000,9.260000,-6.200000,-4.920000,9.930000,16,7,2
2,F00000OTTT,9,1,0,322.70,27.99,3.04,2.71,10.35,0.97,16.87,5.847373,13.80,21.53,14.76,7.83,16.35,11.62,95.92,0.00,2.21,1.87,6.35,22.32,10.82,0.44,11.74,3.37,0.00,17.14,0.00,4.39,23.42,62.38,21.08,11.85,3.33,1.37,40.0,1.018200,0.750000,4.23,8.38,6.99,24.98,4.0,44.0,0.26,0.00,2.83,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,7,2.32,6,30900000.0,6,1.282800e+08,45.43,13.19,18.480000,22.29,-19.51,29.120000,24.700000,-5.990000,17.13,27.23,-9.07,6.64,1.42,3.90,8.82,-7.94,-7.33,4.11,-9.37,9.92,0.800000,6.430000,9.500000,-2.930000,12.530000,6.06000,7.640000,7.340000,-10.010000,-10.820000,9.130000,16,7,3
3,F000005K32,183,2,0,229.67,17.19,1.77,0.75,6.52,2.73,7.77,3.170000,3.04,-0.32,1.86,4.68,17.51,8.65,99.98,0.00,0.21,-0.19,3.01,14.47,13.86,1.73,11.37,10.43,1.39,8.46,3.27,16.32,15.68,30.86,33.09,23.26,11.14,1.66,1106.0,0.550000,0.300000,4.39,9.91,7.06,23.50,2.0,79.0,7.35,0.12,1

In [ ]:
# Splitting Data
X = df_encoded.drop(["rating","ticker"],axis=1)
y = df_encoded["rating"]

# Melakukan splitting terhadap data train dan test (0.2 test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y,random_state = 123)

## Normalisasi dengan Robust Scaler

Normalisasi hanya diimplementasikan kepada kolom numerik (tidak termasuk kolom kategorikal yang dilakukan encoding. Alasan menggunakan Robust Scaler: Data mengandung banyak outliers, Robust Scaler dirancang untuk lebih tahan terhadap outliers.

In [ ]:
# Menggunakan RobustScaler pada data pelatihan (kolom numerik)
scaler_numeric = RobustScaler()

scaled_train = X_train.copy()
scaled_test = X_test.copy()

# Looping untuk melakukan Robust Scaling pada setiap kolom numerik
for col in numerical_cols:
    # Melakukan Robust Scaling pada kolom numerik
    scaled_train[col+'_scaled'] = scaler_numeric.fit_transform(scaled_train[[col]])
    scaled_test[col+'_scaled'] = scaler_numeric.transform(scaled_test[[col]])

# Hapus kolom numerik asli setelah scaling (opsional)
scaled_train.drop(numerical_cols, axis=1, inplace=True)
scaled_test.drop(numerical_cols, axis=1, inplace=True)

scaled_train

,category,equity_style,equity_size,nav_per_share_currency,shareclass_size_currency,fund_size_currency,equity_size_score_scaled,price_prospective_earnings_scaled,price_book_ratio_scaled,price_sales_ratio_scaled,price_cash_flow_ratio_scaled,dividend_yield_factor_scaled,long_term_projected_earnings_growth_scaled,historical_earnings_growth_scaled,sales_growth_scaled,cash_flow_growth_scaled,book_value_growth_scaled,roa_scaled,roe_scaled,roic_scaled,asset_stock_scaled,asset_bond_scaled,asset_cash_scaled,asset_other_scaled,sector_basic_materials_scaled,sector_consumer_cyclical_scaled,sector_financial_services_scaled,sector_real_estate_scaled,sector_consumer_defensive_scaled,sector_healthcare_scaled,sector_utilities_scaled,sector_communication_services_scaled,sector_energy_scaled,sector_industrials_scaled,sector_technology_scaled,market_cap_giant_scaled,market_cap_large_scaled,market_cap_medium_scaled,market_cap_small_scaled,market_cap_micro_scaled,holdings_n_stock_scaled,ongoing_cost_scaled,management_fees_scaled,environmental_score_scaled,social_score_scaled,governance_score_scaled,sustainability_score_scaled,sustainability_rank_scaled,sustainability_percentage_rank_scaled,involvement_abortive_contraceptive_scaled,involvement_alcohol_scaled,involvement_animal_testing_scaled,involvement_controversial_weapons_scaled,involvement_gambling_scaled,involvement_gmo_scaled,involvement_military_contracting_scaled,involvement_nuclear_scaled,involvement_palm_oil_scaled,involvement_pesticides_scaled,involvement_small_arms_scaled,involvement_thermal_coal_scaled,involvement_tobacco_scaled,nav_per_share_scaled,shareclass_size_scaled,fund_size_scaled,fund_trailing_return_ytd_scaled,fund_trailing_return_3years_scaled,fund_trailing_return_5years_scaled,fund_return_2019_scaled,fund_return_2018_scaled,fund_return_2017_scaled,fund_return_2016_scaled,fund_return_2015_scaled,fund_return_2020_q3_scaled,fund_return_2020_q2_scaled,fund_return_2020_q1_scaled,fund_return_2019_q4_scaled,fund_return_2019_q3_scaled,fund_return_2019_q2_scaled,fund_return_2019_q1_scaled,fund_return_2018_q4_scaled,fund_return_2018_q3_scaled,fund_return_2018_q2_scaled,fund_return_2018_q1_scaled,fund_return_2017_q4_scaled,fund_return_2017_q3_scaled,fund_return_2017_q2_scaled,fund_return_2017_q1_scaled,fund_return_2016_q4_scaled,fund_return_2016_q3_scaled,fund_return_2016_q2_scaled,fund_return_2016_q1_scaled,fund_return_2015_q4_scaled,fund_return_2015_q3_scaled,fund_return_2015_q2_scaled,fund_return_2015_q1_scaled,quarters_up_scaled,quarters_down_scaled
22204,65,2,0,8,6,6,-0.183915,-0.482168,-0.629921,-0.538462,-0.606822,0.891753,-0.025890,-0.524242,-0.700000,-0.398261,-0.824766,-0.575931,-0.468788,-0.554371,-4.838375,204.444444,2.625369,9.314286,0.427046,-0.237726,0.477140,0.281991,0.319838,0.145957,0.157518,-0.187500,1.002398,0.331089,-0.284920,0.117910,-0.449577,-0.052063,0.772177,1.942446,-0.510897,-0.235294,-0.049580,0.366972,0.360268,0.381150,0.403240,-1.000000,0.682614,0.363894,0.398876,0.246128,0.582418,0.86,0.04,0.443114,0.875000,0.15,3.409091,1.211538,0.524887,1.609524,0.693931,-0.160892,-0.191548,-0.412669,-0.347267,-0.816512,-0.366975,-0.018358,-0.346153,-1.176647,0.453315,-0.162602,-0.500000,-0.162855,0.202632,-0.140449,-0.446133,-0.391129,0.089888,-0.091877,-0.068254,0.067568,-0.063558,-0.594502,0.250018,-0.201878,-0.174766,-0.553827,-0.514899,-1.075558,0.250677,-0.178519,0.506794,0.194131,0.25,0.000000
15730,91,2,0,16,14,12,-0.201367,-0.887304,0.125984,-0.564103,-0.394973,0.396907,-0.372168,0.823232,0.406122,0.718261,-0.315421,1.060172,1.725826,1.149254,0.084326,0.777778,-0.005900,0.000000,0.247331,-0.078811,-0.138335,-0.355450,0.603239,0.524655,0.735084,-0.188953,-0.510791,-0.470258,0.606671,-0.581643,1.225558,0.602278,-0.211694,0.000000,1.249732,-0.906765,-1.036247,-0.486239,0.456759,-0.207891,-0.632221,0.000000,-0.217386,0.511150,-0.331461,0.582155,0.010989,0.40,0.00,-0.227545,1.755682,0.20,0.000000,0.403846,0.316742,2.180952,-0.372422,0.006351,-0.238101,-0.527361,-0.4

## Yeo-Johnson Transform

Kami juga mencoba mengurangi dampak outliers dengan melakukan transformasi data dengan metode yeo-johnson.
Alasan Menggunakan Yeo-Johnson Transform:
- Data mengandung nilai nol dan nilai negatif
- Lebih tahan dan mampu menangani outliers
- Fleksibel dalam menangani berbagai jenis distribusi data

In [ ]:
from sklearn.preprocessing import PowerTransformer

# Membuat objek PowerTransformer
pt = PowerTransformer(method='yeo-johnson', standardize=True)

# Looping untuk melakukan Yeo-Johnson Transform pada setiap kolom numerik
for col in numerical_cols:
    # Melakukan Yeo-Johnson Transform pada kolom numerik
    col_transformed = pt.fit_transform(scaled_train[[col+'_scaled']])
    scaled_train[col+'_transformed'] = col_transformed

    col_transformed_test = pt.transform(scaled_test[[col+'_scaled']])
    scaled_test[col+'_transformed'] = col_transformed_test

    # Hapus kolom numerik asli setelah scaling (opsional)
    scaled_train.drop(col+'_scaled', axis=1, inplace=True)
    scaled_test.drop(col+'_scaled', axis=1, inplace=True)

scaled_train

,category,equity_style,equity_size,nav_per_share_currency,shareclass_size_currency,fund_size_currency,equity_size_score_transformed,price_prospective_earnings_transformed,price_book_ratio_transformed,price_sales_ratio_transformed,price_cash_flow_ratio_transformed,dividend_yield_factor_transformed,long_term_projected_earnings_growth_transformed,historical_earnings_growth_transformed,sales_growth_transformed,cash_flow_growth_transformed,book_value_growth_transformed,roa_transformed,roe_transformed,roic_transformed,asset_stock_transformed,asset_bond_transformed,asset_cash_transformed,asset_other_transformed,sector_basic_materials_transformed,sector_consumer_cyclical_transformed,sector_financial_services_transformed,sector_real_estate_transformed,sector_consumer_defensive_transformed,sector_healthcare_transformed,sector_utilities_transformed,sector_communication_services_transformed,sector_energy_transformed,sector_industrials_transformed,sector_technology_transformed,market_cap_giant_transformed,market_cap_large_transformed,market_cap_medium_transformed,market_cap_small_transformed,market_cap_micro_transformed,holdings_n_stock_transformed,ongoing_cost_transformed,management_fees_transformed,environmental_score_transformed,social_score_transformed,governance_score_transformed,sustainability_score_transformed,sustainability_rank_transformed,sustainability_percentage_rank_transformed,involvement_abortive_contraceptive_transformed,involvement_alcohol_transformed,involvement_animal_testing_transformed,involvement_controversial_weapons_transformed,involvement_gambling_transformed,involvement_gmo_transformed,involvement_military_contracting_transformed,involvement_nuclear_transformed,involvement_palm_oil_transformed,involvement_pesticides_transformed,involvement_small_arms_transformed,involvement_thermal_coal_transformed,involvement_tobacco_transformed,nav_per_share_transformed,shareclass_size_transformed,fund_size_transformed,fund_trailing_return_ytd_transformed,fund_trailing_return_3years_transformed,fund_trailing_return_5years_transformed,fund_return_2019_transformed,fund_return_2018_transformed,fund_return_2017_transformed,fund_return_2016_transformed,fund_return_2015_transformed,fund_return_2020_q3_transformed,fund_return_2020_q2_transformed,fund_return_2020_q1_transformed,fund_return_2019_q4_transformed,fund_return_2019_q3_transformed,fund_return_2019_q2_transformed,fund_return_2019_q1_transformed,fund_return_2018_q4_transformed,fund_return_2018_q3_transformed,fund_return_2018_q2_transformed,fund_return_2018_q1_transformed,fund_return_2017_q4_transformed,fund_return_2017_q3_transformed,fund_return_2017_q2_transformed,fund_return_2017_q1_transformed,fund_return_2016_q4_transformed,fund_return_2016_q3_transformed,fund_return_2016_q2_transformed,fund_return_2016_q1_transformed,fund_return_2015_q4_transformed,fund_return_2015_q3_transformed,fund_return_2015_q2_transformed,fund_return_2015_q1_transformed,quarters_up_transformed,quarters_down_transformed
22204,65,2,0,8,6,6,-0.195028,-0.636224,-1.238277,-1.120579,-1.047367,0.976059,-0.010743,-0.501761,-0.646932,-0.336015,-0.805466,-0.659561,-0.571031,-0.608710,-1.327431,0.574531,0.794667,1.733714,0.568437,-0.145488,0.518233,0.494961,0.445872,0.401627,-0.056170,-0.095760,1.117866,0.470233,-0.260051,0.243655,-0.367614,-0.099290,0.790378,0.612506,-2.194375,-0.391262,-0.147206,0.461499,0.368676,0.448339,0.470136,-1.115553,1.052295,0.548962,0.592527,0.404135,0.636716,0.856567,2.399001,0.611406,0.978593,2.577836,1.624325,1.180151,0.762819,1.319425,0.747312,-0.954727,-0.787369,-0.401587,-0.418230,-0.849008,-0.472752,-0.097532,-0.288302,-1.169805,0.192651,-0.212265,-0.562099,-0.215554,0.243291,-0.162957,-0.510771,-0.477349,0.108998,-0.230140,-0.165411,0.092999,-0.041957,-0.716078,0.219523,-0.249098,-0.143394,-0.612109,-0.455458,-0.937026,0.275745,-0.257420,0.505038,0.102940,0.402990,0.148675
15730,91,2,0,16,14,12,-0.214541,-1.431794,0.168431,-1.189135,-0.627187,0.502843,-0.439050,0.735901,0.2568

## Feature Selection

Selain menetapkan missing values treshold sebesar 50%, kami juga menggunakan SelectKBest, dengan k sebanyak 50: Didapat dengan melakukan eksperimen, k=50 memiliki running time yang tidak terlalu lama dan akurasi relatif tinggi.

In [ ]:
# Feature Selection

select = SelectKBest(score_func=f_classif, k=50)
z = select.fit_transform(scaled_train,y_train)

filter = select.get_support()
selected_features  = array(scaled_train.columns)

# Drop columns from X_train that are not among the selected best 30 features
selected_columns = scaled_train.columns[filter]
X_train_selected = scaled_train[selected_columns]
scaled_test = scaled_test[selected_columns]

X_train_selected

,equity_style,equity_size_score_transformed,price_prospective_earnings_transformed,price_book_ratio_transformed,price_sales_ratio_transformed,price_cash_flow_ratio_transformed,dividend_yield_factor_transformed,long_term_projected_earnings_growth_transformed,historical_earnings_growth_transformed,sales_growth_transformed,book_value_growth_transformed,roa_transformed,roe_transformed,roic_transformed,sector_financial_services_transformed,sector_energy_transformed,sector_technology_transformed,market_cap_giant_transformed,market_cap_small_transformed,market_cap_micro_transformed,ongoing_cost_transformed,management_fees_transformed,environmental_score_transformed,sustainability_score_transformed,sustainability_rank_transformed,sustainability_percentage_rank_transformed,involvement_thermal_coal_transformed,shareclass_size_transformed,fund_size_transformed,fund_trailing_return_ytd_transformed,fund_trailing_return_3years_transformed,fund_trailing_return_5years_transformed,fund_return_2019_transformed,fund_return_2018_transformed,fund_return_2017_transformed,fund_return_2015_transformed,fund_return_2020_q3_transformed,fund_return_2020_q2_transformed,fund_return_2020_q1_transformed,fund_return_2019_q3_transformed,fund_return_2019_q2_transformed,fund_return_2019_q1_transformed,fund_return_2018_q2_transformed,fund_return_2018_q1_transformed,fund_return_2017_q4_transformed,fund_return_2017_q2_transformed,fund_return_2017_q1_transformed,fund_return_2015_q4_transformed,quarters_up_transformed,quarters_down_transformed
22204,2,-0.195028,-0.636224,-1.238277,-1.120579,-1.047367,0.976059,-0.010743,-0.501761,-0.646932,-0.805466,-0.659561,-0.571031,-0.608710,0.518233,1.117866,-0.260051,0.243655,0.790378,0.612506,-0.391262,-0.147206,0.461499,0.470136,-1.115553,1.052295,0.762819,-0.954727,-0.787369,-0.401587,-0.418230,-0.849008,-0.472752,-0.097532,-0.288302,0.192651,-0.212265,-0.562099,-0.215554,-0.162957,-0.510771,-0.477349,-0.165411,0.092999,-0.041957,0.219523,-0.249098,0.275745,0.402990,0.148675
15730,2,-0.214541,-1.431794,0.168431,-1.189135,-0.627187,0.502843,-0.439050,0.735901,0.256884,-0.388012,1.552169,2.260816,1.383762,0.006139,-1.231862,0.869574,-0.718490,-0.793630,-0.496514,-1.653036,-1.714451,-0.136653,-0.516969,-0.148927,-0.300546,0.535406,-0.372702,-0.959308,-0.524048,-0.496092,0.039521,-0.812284,0.172916,0.083798,-0.082921,-0.449946,-0.600896,0.071297,0.849995,-1.178208,-0.758148,0.020099,0.135824,0.644168,-0.038246,0.079418,-0.002555,-1.530246,-1.320764
18872,1,0.686147,0.317596,1.042710,0.599422,0.873703,-1.614154,0.896137,0.576965,0.007110,1.218634,-0.072756,0.061825,-0.743687,-0.290139,-1.248463,-1.198799,0.442834,-0.932188,-0.496514,0.008243,0.335592,-1.103442,-0.058666,-1.115553,0.981227,-1.071961,-0.876001,1.154249,0.502673,0.987561,0.997135,0.785836,1.208978,-0.085176,-0.082921,0.062191,-0.320860,0.643083,0.943075,0.467387,0.519269,0.923126,-0.740355,-1.203121,1.014600,0.417018,0.808701,0.402990,-0.339331
4055,1,-1.443478,1.509817,1.108159,0.566174,0.725091,-0.857162,1.573355,0.158584,0.475012,1.218634,-0.104225,0.126117,-0.148141,-0.466579,-1.248463,0.320069,-1.645793,1.341918,0.299252,-0.210971,-0.127918,-0.852404,-0.142761,-0.148927,-0.220471,-1.071961,0.791526,1.221920,0.795353,0.729652,1.165173,0.822980,-0.076610,0.083798,-0.082921,1.474640,1.279121,-0.267540,-1.418321,1.461811,0.458153,0.344828,0.890227,0.190447,2.806022,1.015109,1.712659,1.008875,-0.339331
9924,1,2.156631,0.806681,0.995343,1.036093,0.876802,-1.273660,0.929546,0.852404,1.311832,2.101516,0.838118,0.499779,1.362937,1.218504,1.378881,-0.232304,2.003703,-0.983463,-0.496514,1.162794,1.183316,0.338410,0.620994,-0.148927,0.472170,-1.071961,0.610547,0.477952,0.569478,0.185569,1.006730,-0.592820,0.034265,1.746496,-0.448195,0.751366,0.311973,-0.035608,-0.887761,-0.219046,0.265622,-0.635165,0.322550,1.356943,0.866065,1.420775,0.147023,-0.449709,1.118168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

## Imbalance Handling

Kami menguji dan mengidentifikasi bahwa data termasuk imbalance data. Kami menanganinya dengan menggunakan Borderline SMOTE. Borderline SMOTE memusatkan pembuatan sampel sintetis pada batas kelas, sehingga mempertahankan struktur data asli dengan lebih baik.

In [ ]:
import numpy as np

# Menghitung jumlah setiap kelas pada set pelatihan
unique_classes, class_counts = np.unique(y_train, return_counts=True)

# Menampilkan jumlah kelas
for cls, count in zip(unique_classes, class_counts):
    print(f'Class {cls}: {count} samples')

Class 0: 1094 samples
Class 1: 3310 samples
Class 2: 6581 samples
Class 3: 4673 samples
Class 4: 2278 samples


In [ ]:
# Inisiasi oversampler dengan metode Borderline SMOTE
borderline_smote = BorderlineSMOTE()

# Melakukan resampling
X_borderline_smote, y_borderline_smote = borderline_smote.fit_resample(X_train_selected, y_train)

# Melihat distribusi kelas pada dataset
counter_y_borderline_smote = Counter(y_borderline_smote)
print(counter_y_borderline_smote)

Counter({1: 6581, 2: 6581, 3: 6581, 4: 6581, 0: 6581})


# Pembuatan Model

Selanjutnya, Saya mencoba untuk menggunakan beberapa model untuk mendapatkan model dengan hasil yang terbaik

## XGBoost

In [ ]:
import xgboost as xgb

# Inisialisasi model XGBoost
xgb_model = xgb.XGBClassifier(n_estimators=200, max_depth=8,min_child_weight=5,subsample=1)
xgb_model.fit(X_borderline_smote, y_borderline_smote)

y_pred_resampled = xgb_model.predict(scaled_test)
evaluate_classifier_performance(y_pred_resampled, y_test)

Hasil Evaluasi berdasarkan classification report 

              precision    recall  f1-score   support

           0       0.81      0.82      0.81       274
           1       0.83      0.80      0.81       828
           2       0.82      0.85      0.84      1645
           3       0.80      0.80      0.80      1168
           4       0.87      0.83      0.85       569

    accuracy                           0.82      4484
   macro avg       0.83      0.82      0.82      4484
weighted avg       0.82      0.82      0.82      4484



Confusion Matrix



prediction,0,1,2,3,4
actual,,,,,
0,225,32,17,0,0
1,45,661,110,11,1
2,6,98,1400,137,4
3,2,5,159,935,67
4,1,4,11,79,474




Butuh informasi lebih lengkap? silakan simak di bawah ini : 
Accuracy Average: 0.8240410347903657
F1 Macro Average: 0.8232807034126086
F1 Micro Average: 0.8240410347903657
Precision Macro Average: 0.8260931818448306
Precision Micro Average: 0.8240410347903657
Recall Macro Average: 0.8208190024331259
Recall Micro Average: 0.8240410347903657



## Decission Tree

In [ ]:
# Membuat model decision tree
dtr_model = DecisionTreeClassifier()
dtr_model.fit(X_borderline_smote, y_borderline_smote)

DecisionTreeClassifier()

In [ ]:
# Memprediksi data testing
y_pred = dtr_model.predict(scaled_test)

# Menampilkan metrics evaluasi
evaluate_classifier_performance(y_pred, y_test)

Hasil Evaluasi berdasarkan classification report 

              precision    recall  f1-score   support

           0       0.67      0.75      0.71       274
           1       0.72      0.75      0.73       828
           2       0.78      0.73      0.76      1645
           3       0.72      0.73      0.73      1168
           4       0.75      0.77      0.76       569

    accuracy                           0.74      4484
   macro avg       0.73      0.75      0.74      4484
weighted avg       0.74      0.74      0.74      4484



Confusion Matrix



prediction,0,1,2,3,4
actual,,,,,
0,206,47,15,3,3
1,55,623,110,28,12
2,32,163,1201,215,34
3,10,25,179,857,97
4,5,11,30,83,440




Butuh informasi lebih lengkap? silakan simak di bawah ini : 
Accuracy Average: 0.741971454058876
F1 Macro Average: 0.737502737498988
F1 Micro Average: 0.7419714540588761
Precision Macro Average: 0.7283215589177223
Precision Micro Average: 0.741971454058876
Recall Macro Average: 0.7482701612129856
Recall Micro Average: 0.741971454058876



## Random Forest

In [ ]:
# Membuat model random forest dengan param
rf_model = RandomForestClassifier(criterion='entropy', max_depth=None, n_estimators=300, min_samples_split=2)
rf_model.fit(X_borderline_smote, y_borderline_smote)

RandomForestClassifier(criterion='entropy', n_estimators=300)

In [ ]:
# Memprediksi data testing
predicted_rf = rf_model.predict(scaled_test)

# Menampilkan metrics evaluasi
evaluate_classifier_performance(predicted_rf, y_test)

Hasil Evaluasi berdasarkan classification report 

              precision    recall  f1-score   support

           0       0.81      0.81      0.81       274
           1       0.80      0.79      0.79       828
           2       0.83      0.83      0.83      1645
           3       0.80      0.80      0.80      1168
           4       0.85      0.85      0.85       569

    accuracy                           0.82      4484
   macro avg       0.82      0.82      0.82      4484
weighted avg       0.82      0.82      0.82      4484



Confusion Matrix



prediction,0,1,2,3,4
actual,,,,,
0,223,39,12,0,0
1,50,657,108,9,4
2,2,124,1363,150,6
3,0,3,152,936,77
4,0,2,6,78,483




Butuh informasi lebih lengkap? silakan simak di bawah ini : 
Accuracy Average: 0.8166815343443354
F1 Macro Average: 0.8169310336908071
F1 Micro Average: 0.8166815343443354
Precision Macro Average: 0.8166372431011434
Precision Micro Average: 0.8166815343443354
Recall Macro Average: 0.8172291621169183
Recall Micro Average: 0.8166815343443354



## Support Vector Machine (SVM)

In [ ]:
from sklearn.svm import SVC

svm_classifier = SVC()

svm_classifier.fit(X_borderline_smote, y_borderline_smote)

svm_predicted = svm_classifier.predict(scaled_test)

evaluate_classifier_performance(svm_predicted, y_test)

Hasil Evaluasi berdasarkan classification report 

              precision    recall  f1-score   support

           0       0.58      0.81      0.67       274
           1       0.59      0.68      0.63       828
           2       0.75      0.60      0.66      1645
           3       0.67      0.63      0.65      1168
           4       0.65      0.84      0.73       569

    accuracy                           0.66      4484
   macro avg       0.65      0.71      0.67      4484
weighted avg       0.68      0.66      0.66      4484



Confusion Matrix



prediction,0,1,2,3,4
actual,,,,,
0,222,38,13,1,0
1,135,563,109,19,2
2,24,318,979,279,45
3,2,35,188,735,208
4,1,6,18,66,478




Butuh informasi lebih lengkap? silakan simak di bawah ini : 
Accuracy Average: 0.6639161462979483
F1 Macro Average: 0.6700416449443926
F1 Micro Average: 0.6639161462979483
Precision Macro Average: 0.6467846720766264
Precision Micro Average: 0.6639161462979483
Recall Macro Average: 0.7109317135453054
Recall Micro Average: 0.6639161462979483



## K-Nearest Neighbors (KNN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_classifier = KNeighborsClassifier()

knn_classifier.fit(X_borderline_smote, y_borderline_smote)

knn_predicted = knn_classifier.predict(scaled_test)

evaluate_classifier_performance(knn_predicted, y_test)

Hasil Evaluasi berdasarkan classification report 

              precision    recall  f1-score   support

           0       0.66      0.81      0.73       274
           1       0.63      0.74      0.68       828
           2       0.80      0.64      0.71      1645
           3       0.70      0.73      0.72      1168
           4       0.73      0.83      0.77       569

    accuracy                           0.72      4484
   macro avg       0.70      0.75      0.72      4484
weighted avg       0.73      0.72      0.72      4484



Confusion Matrix



prediction,0,1,2,3,4
actual,,,,,
0,223,37,6,5,3
1,90,614,97,19,8
2,22,277,1053,259,34
3,2,38,143,851,134
4,1,3,18,75,472




Butuh informasi lebih lengkap? silakan simak di bawah ini : 
Accuracy Average: 0.7165477252453167
F1 Macro Average: 0.7225849007372128
F1 Micro Average: 0.7165477252453167
Precision Macro Average: 0.7043753152794595
Precision Micro Average: 0.7165477252453167
Recall Macro Average: 0.7507314922241213
Recall Micro Average: 0.7165477252453167



## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

naive_bayes_classifier = GaussianNB()

naive_bayes_classifier.fit(X_borderline_smote, y_borderline_smote)

naive_bayes_predicted = naive_bayes_classifier.predict(scaled_test)

evaluate_classifier_performance(naive_bayes_predicted, y_test)

Hasil Evaluasi berdasarkan classification report 

              precision    recall  f1-score   support

           0       0.20      0.67      0.31       274
           1       0.30      0.30      0.30       828
           2       0.49      0.28      0.35      1645
           3       0.38      0.21      0.27      1168
           4       0.30      0.62      0.41       569

    accuracy                           0.33      4484
   macro avg       0.33      0.42      0.33      4484
weighted avg       0.38      0.33      0.33      4484



Confusion Matrix



prediction,0,1,2,3,4
actual,,,,,
0,183,61,15,7,8
1,317,247,134,41,89
2,268,344,456,249,328
3,111,136,275,249,397
4,21,24,57,113,354




Butuh informasi lebih lengkap? silakan simak di bawah ini : 
Accuracy Average: 0.33206958073148973
F1 Macro Average: 0.32889948302595234
F1 Micro Average: 0.33206958073148973
Precision Macro Average: 0.3346091410812227
Precision Micro Average: 0.33206958073148973
Recall Macro Average: 0.4157450163648175
Recall Micro Average: 0.33206958073148973



# Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score, KFold

# Define the models
models = [
    ('XGBoost', xgb.XGBClassifier(n_estimators=200, max_depth=8, min_child_weight=5, subsample=1)),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Random Forest', RandomForestClassifier(criterion='entropy', max_depth=None, n_estimators=300, min_samples_split=2)),
    ('SVM', SVC()),
    ('KNN', KNeighborsClassifier()),
    ('Naive Bayes', GaussianNB())
]

k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

model_names = []
mean_accuracies = []

for name, model in models:
    cross_val_result = cross_val_score(model, X_borderline_smote, y_borderline_smote, cv=k_fold, scoring='accuracy')
    mean_accuracy = cross_val_result.mean()

    model_names.append(name)
    mean_accuracies.append(mean_accuracy)

    print(f'{name} - Cross-Validation result: {cross_val_result}')
    print(f'{name} - Mean Accuracy: {mean_accuracy}')
    print()

XGBoost - Cross-Validation result: [0.90670111 0.91049992 0.9089804  0.90578939 0.91566631]
XGBoost - Mean Accuracy: 0.909527427442638

Decision Tree - Cross-Validation result: [0.80048625 0.80276554 0.80534873 0.79410424 0.80306944]
Decision Tree - Mean Accuracy: 0.8011548396900168

Random Forest - Cross-Validation result: [0.9032062  0.90806868 0.90822064 0.90047105 0.90730892]
Random Forest - Mean Accuracy: 0.9054550980094211

SVM - Cross-Validation result: [0.76037076 0.76979182 0.76416958 0.75642    0.76933597]
SVM - Mean Accuracy: 0.7640176265005318

KNN - Cross-Validation result: [0.83513144 0.83725878 0.83649901 0.83649901 0.83695487]
KNN - Mean Accuracy: 0.8364686217900015

Naive Bayes - Cross-Validation result: [0.40844856 0.41528643 0.4137669  0.41042395 0.41072785]
Naive Bayes - Mean Accuracy: 0.41173074000911714



# Hasil Analisis

Model yang tahan terhadap outliers seperti XGBoost dan Random Forest memiliki akurasi yang relatif lebih tinggi. Sedangkan model yang sensitif terhadap outliers seperti KNN dan naive bayes memiliki akurasi yang relatif rendah.